In [1]:
import numpy as np
from pathlib import Path as pt

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import clone

In [2]:
base_loc = pt('/Users/aravindhnivas/Documents/ML-properties/[PHYSICAL CONSTANTS OF ORGANIC COMPOUNDS]/tmp_C_processed_data/analysis_data/filtered/tmpC_topelements_processed_data/embedded_vectors')

# embeddings = 'mol2vec_embeddings'
embeddings = 'VICGAE_embeddings'
processed_vec_dir = base_loc / f'processed_{embeddings}'
X = np.load(processed_vec_dir / 'processed.X.npy', allow_pickle=True)
y = np.load(processed_vec_dir / 'processed.y.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((5760, 32), (5760,), (1440, 32), (1440,))

In [3]:
gbr_model = GradientBoostingRegressor()
xgb_model = XGBRegressor(verbosity=0)
cat_model = CatBoostRegressor(verbose=0)
lgb_model = LGBMRegressor(verbose=-1)

In [4]:
def train_model(model, X_train, y_train):
    cloned_model = clone(model)
    cloned_model.fit(X_train, y_train)
    return

In [5]:
%%timeit
train_model(gbr_model, X_train, y_train)

4.3 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
train_model(xgb_model, X_train, y_train)

263 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
train_model(cat_model, X_train, y_train)

1.6 s ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
train_model(lgb_model, X_train, y_train)

541 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Apple M2 Pro
32 GB, 10-core CPU

### Mol2vec

GBR: 40.6 s ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

CATBOOST: 7.94 s ± 411 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

XGBOOST: 1.24 s ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

LGBM: 861 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

### VICGAE

GBR: 4.3 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

CATBOOST: 1.6 s ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

XGBOOST: 263 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

LGBM: 541 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)